#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=15994 sha256=4c70a6abbd7046727af3bba3b09906c634dc7a7a5bfcddede652616d3e79e7e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8l5emij/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [1]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree

2023-09-09 18:53:06.509498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 18:53:06.881838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 18:53:06.884532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 18:53:08.570351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##Get latest file URLS and Current Date Ranges

In [2]:
latest_URLS = Boston311LogReg.get311URLs()

In [3]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp69ksx7xb.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [4]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")

print(today_datestring, thirty_days_ago_datestring)

2023-09-09 2023-08-10


In [5]:
#set model folder constant
MODEL_FOLDER = './daily_models'

##Define several models

In [6]:
linear_tree_model = Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':'2023-05-22'},
                            files_dict=latest_URLS)

In [10]:
logistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [11]:
logistic_tree_model = Boston311EventDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

##Train several models

In [12]:
logistic_tree_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-08 17:54:14.080993
Testing accuracy: 0.9591651986690155
Ending Training at 2023-09-08 17:54:38.895549
Training took 0:00:24.814556


In [18]:
import os

# Define the directory path
dir_path = MODEL_FOLDER+'/logtree'

# Create the directory
# If the directory does not exist, it will be created. If it exists, this operation will do nothing.
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

logistic_tree_model.save(dir_path,'logtree','logtreeproperties')

In [19]:
logistic_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-08 18:00:59.445552
Epoch 1/10


2023-09-08 18:00:59.951862: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 88675620 exceeds 10% of free system memory.


8175/8175 [==============================] - 6s 652us/step - loss: 0.1856 - accuracy: 0.9335 - val_loss: 0.1195 - val_accuracy: 0.9504
Epoch 2/10
8175/8175 [==============================] - 5s 646us/step - loss: 0.1112 - accuracy: 0.9533 - val_loss: 0.1054 - val_accuracy: 0.9552
Epoch 3/10
8175/8175 [==============================] - 5s 632us/step - loss: 0.1041 - accuracy: 0.9564 - val_loss: 0.1016 - val_accuracy: 0.9575
Epoch 4/10
8175/8175 [==============================] - 5s 636us/step - loss: 0.1017 - accuracy: 0.9576 - val_loss: 0.1000 - val_accuracy: 0.9583
Epoch 5/10
8175/8175 [==============================] - 5s 632us/step - loss: 0.1006 - accuracy: 0.9579 - val_loss: 0.0992 - val_accuracy: 0.9587
Epoch 6/10
8175/8175 [==============================] - 5s 640us/step - loss: 0.1001 - accuracy: 0.9579 - val_loss: 0.0988 - val_accuracy: 0.9587
Epoch 7/10
8175/8175 [==============================] - 5s 630us/step - loss: 0.0997 - accuracy: 0.9582 - val_loss: 0.0985 - val_accura

In [20]:
# Define the directory path
dir_path = MODEL_FOLDER+'/logreg'

# Create the directory
# If the directory does not exist, it will be created. If it exists, this operation will do nothing.
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    
logistic_model.save(dir_path,'logreg','logregproperties')

In [21]:
import gc
gc.collect()

2509

In [7]:
linear_tree_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-09 18:54:47.287012


In [ ]:
# Define the directory path
dir_path = MODEL_FOLDER+'/linreg'

# Create the directory
# If the directory does not exist, it will be created. If it exists, this operation will do nothing.
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

linear_tree_model.save(dir_path,'linreg','linregproperties')